<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/Agent_SDK_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent SDK Demo

Demonstriert die Verwendung von OpenAI's Agent SDK.

**Setup**

In [ ]:
!pip install -qU openai-agents==0.0.3

In [ ]:
import os
import json
from agents import set_tracing_export_api_key

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

set_tracing_export_api_key(os.environ.get('OPENAI_API_KEY'))

# Agent definieren

In [ ]:
from agents import Agent, Runner

agent = Agent(
    name="Assistent",
    instructions="Du bist ein hilfreicher Assistent.",
    model="gpt-4o-mini",
)

#Agent ausführen

In [ ]:
result = await Runner.run(
    starting_agent=agent,
    input="Stelle ein einfaches Mittagsmenü zusammen."
)
output = result.final_output

# Markdown ausgeben
from IPython.display import Markdown
display(Markdown(output))

#Streaming
Die Ausgabe kann auch gestreamed werden.

In [ ]:
from openai.types.responses import ResponseTextDeltaEvent

response = Runner.run_streamed(
    starting_agent=agent,
    input="Erkläre die Grundlagen von generativer KI"
)

async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

#Funktionen einbinden

In [ ]:
einkaufsliste = []

from agents import function_tool

@function_tool
def add_to_shopping_list(article: str):
    einkaufsliste.append(article)
    print(f"### add_to_shopping_list: '{article}' wurde zur Einkaufsliste hinzugefügt. ###")
    #print("Aktuelle Einkaufsliste:", einkaufsliste)


Nun können wir die Funktion als Tool beim Agent registrieren...

In [ ]:
agent = Agent(
    name="Assistant",
    instructions=("Du bist ein hilfreicher Assistent."),
    model="gpt-4o-mini",
    tools=[add_to_shopping_list]
)

... und die Abfrage starten

In [ ]:
response = await Runner.run(
    starting_agent=agent,
    input="Füge Eier zur Einkaufsliste hinzu!"
)
print(response.final_output)


#Tool einbinden
WebSearch ist ein vorgefertigtes Tool on OpenAI. Es ermöglicht für das Erledigen der Aufgabe eine Web-Suche zu verwenden.

In [ ]:
from agents import WebSearchTool

web_agent = Agent(
    name="Web Assistant",
    instructions="Du bist ein Assisten, der aktuelle Nachrichten aus Internet ermittelt.",
    output_type=str,
    tools=[WebSearchTool()],
)
output = await Runner.run(
    starting_agent=web_agent,
    input="Ermittle die Ergebnisse des 3. Spieltages 2025 der deutschen Fussball-Bundesliga.",
)
from IPython.display import Markdown
display(Markdown(output.final_output))

#Conversation

In [ ]:
# Agent mit einer Anweisung aufrufen
result = await Runner.run(
    starting_agent=agent,
    input="Berechne die Quadratwurzel aus 10."
)

print(f'1. Durchlauf: {result.final_output}')
print(json.dumps(result.to_input_list(), indent=4, ensure_ascii=False))

# Agent mit dem vorherigen Ergebnis und zusätzlicher Abfrage erneut aufrufen
result = await Runner.run(
    starting_agent=agent,
    input=result.to_input_list() + [
        {"role": "user", "content": "Generiere für die Berechnung eine Python-Funktion."}
    ]
)
print(f'2. Durchlauf: {result.final_output}')

#Handoffs
Handoffs bezeichnet die Delegation der Kontrolle von einem Agent zum nächsten. So kann z.B. flexibel anhand des Prompts entschieden werden, welcher Agent die Aufgabe übernehmen soll.

In [ ]:
from openai.types.responses import ResponseTextDeltaEvent

@function_tool
def record_travel_costs(amount: float):
    return "Reisekosten wurden verbucht."

@function_tool
def record_fuel_costs(amount: float):
    return "Tankkosten wurden verbucht."

@function_tool
def record_expenses(amount: float):
    return "Spesen wurden verbucht."

travel_cost_agent = Agent(
    name="Travel Cost Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Reisekosten. Nutze dafür die bereitgestellten Tool.",
    handoff_description="Du bist ein Buchhalter und bearbeitest Reisekosten. Nutze dafür die bereitgestellten Tool.",
    tools=[record_travel_costs],
)
fuel_cost_agent = Agent(
    name="Fuel Cost Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Tankkosten. Nutze dafür die bereitgestellten Tool.",
    handoff_description="Du bist ein Buchhalter und bearbeitest Tankkosten. Nutze dafür die bereitgestellten Tool.",
    tools=[record_fuel_costs],
)
expenses_agent = Agent(
    name="Expenses Agent",
    instructions="Du bist ein Buchhalter und bearbeitest Spesenrechnungen. Nutze dafür die bereitgestellten Tool.",
    handoff_description="Du bist ein Buchhalter und bearbeitest Spesenrechnungen. Nutze dafür die bereitgestellten Tool.",
    tools=[record_expenses],
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="Du routest den Benutzer zu einem passenden Agent. Stelle bitte keine Rückfragen an den Benutzer.",
    handoffs=[fuel_cost_agent, travel_cost_agent, expenses_agent]
)

user_query = "Verbuche bitte meine Reisekosten von €122."
#user_query = "Verbuche bitte meine Tankkosten von €54."
#user_query = "Verbuche bitte meine Spesen von €67."
print("User:", user_query)

output = Runner.run_streamed(
    starting_agent=triage_agent,
    input=user_query,
)

async for event in output.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        print(f"> Aktueller Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        if event.name == "tool_called":
            print()
            print(f"> Tool aufgerufen, Name: {event.item.raw_item.name}")
            print(f"> Tool aufgerufen, Args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            print(f"> Tool Ausgabe: {event.item.raw_item['output']}")

#Agents als Tools
Mehrere Agents können sich auch eine große Aufgabe teilen. Hierzu bricht man die Aufgabe in Teilaufgaben herunter und weist diese den Agenten zu. Der Hauptangent bekommt die Gesamtaufgabe übergeben und routed automatisch an die untergeordneten Agenten weiter. Hierzu werden diese als Tools dem Hauptagenten zugewiesen.

In [ ]:
from agents import Agent, Runner, ItemHelpers, MessageOutputItem
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,
    ResponseCreatedEvent,
)
dessert_agent = Agent(
    name="Nachspeisen-Koch",
    instructions="Du bist ein Koch, der auf Nachspeisen spezialisiert ist. Beginne Deine Ausgabe immer mit dem Wort '[NACHSPEISE]'.",
    handoff_description="Erstellt Rezepte für Nachspeisen.",
)
hauptgang_agent = Agent(
    name="Hauptgang-Koch",
    instructions="Du bist ein Koch, der auf Hauptgänge spezialisiert ist. Beginne Deine Ausgabe immer mit dem Wort '[HAUPTGANG]'.",
    handoff_description="Erstellt Rezepte für Hauptgänge.",
)
vorspeisen_agent = Agent(
    name="Vorspeisen-Koch",
    instructions="Du bist ein Koch, der auf Vorspeisen spezialisiert ist. Beginne Deine Ausgabe immer mit dem Wort '[VORSPEISE]'.",
    handoff_description="Erstellt Rezepte für Vorspeisen.",
)

rezept_agent = Agent(
    name="Rezept Assistant",
    instructions="""
    Du bist ein Assisten, der bei der Erstellung von Rezepten hilft. Du lieferst jedoch nur die Idee für Vorspeise, Hauptgang und Nachspeise.
    Du verwendest die dir zur Verfügung gestellten Tools zur Erstellung der Rezepte für den jeweiligen Gang.
    Wenn du nach mehreren Gängen gefragt wirst, rufst du die entsprechenden Tools der Reihe nach auf.
    Du generierst niemals eigenständig ein Rezept, sondern nutzt stets die bereitgestellten Tools.
    """,
    tools=[
        vorspeisen_agent.as_tool(
            tool_name="Vorspeise",
            tool_description="Liefert das Rezept für die Vorspeise.",
        ),
        hauptgang_agent.as_tool(
            tool_name="Hauptgang",
            tool_description="Liefert das Rezept für den Hauptgang.",
        ),
        dessert_agent.as_tool(
            tool_name="Dessert",
            tool_description="Liefert das Rezept für die Nachspeise.",
        ),
    ],
)

output = Runner.run_streamed(
    starting_agent=rezept_agent,
    input="Generiere ein Mittagsmenü mit Vorspeise, Hauptgang und Nachspeise. Gib mir bitte jeweis die vollständigen Rezepte.",
)

async for event in output.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        print(f"> Aktueller Agent: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        if event.name == "tool_called":
            print()
            print(f"> Tool aufgerufen, Name: {event.item.raw_item.name}")
            print(f"> Tool aufgerufen, Args: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            print(f"> Tool Ausgabe: {event.item.raw_item['output']}")

print(f"""
  \n\n---------------------------\n\n
  Gesamtergebnis:\n{output.final_output}
""")


#Guardrails

Guardrails können für Ein- und Ausgaben definiert werden. Mit ihnen können unerwünschte Inhalte aus der Kommunikation herausgefiltert werden.

In [ ]:
import asyncio
from agents import (
    Agent,
    Runner,
    GuardrailFunctionOutput,
    RunContextWrapper,
    input_guardrail,
    InputGuardrailTripwireTriggered
)
from pydantic import BaseModel

# Struktur für die Guardrail-Prüfung
class GuardrailCheck(BaseModel):
    is_triggered: bool
    reason: str

# Agent für die Guardrail-Prüfung
politics_agent = Agent(
    name="Politik Checker",
    instructions="Überprüft, ob der Benutzer nach politischen Themen fragt. Gib eine Begründung auf deutsch aus.",
    output_type=GuardrailCheck,
)

# Guardrail-Funktion, die den Agenten aufruft
@input_guardrail
async def politics_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str,
) -> GuardrailFunctionOutput:
    response = await Runner.run(starting_agent=politics_agent, input=input)
    return GuardrailFunctionOutput(
        output_info=response.final_output,
        tripwire_triggered=response.final_output.is_triggered,
    )

# Übergeordneter Agent, der die Guardrail-Funktion verwendet
agent = Agent(
    name="Assistant",
    instructions=("Du bist ein hilfreicher Assistent."
    ),
    model="gpt-4o-mini",
    input_guardrails=[politics_guardrail],
)

# Testanfrage
try:
    query = "Was hälst Du von der neuen Bundesregierung?"
    result = await Runner.run(starting_agent=agent, input=query)
    print(result.final_output)
except InputGuardrailTripwireTriggered as e:
    print(f"Guardrail wurde ausgelöst. Grund: {e.guardrail_result.output.output_info.reason}")
